In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from MulticoreTSNE import MulticoreTSNE as TSNE
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

initialize(version_base=None, config_path="configs")
cfg = compose(config_name="config.yaml")

hydra.initialize()

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
from dotenv import load_dotenv
import wandb
import os

load_dotenv()
wandb.login(key=os.getenv("WANDB_KEY"))

api = wandb.Api()

True

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: g-hojatkarami (hokarami). Use `wandb login --relogin` to force relogin


True

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm

# Load Data

In [7]:
pd.options.mode.copy_on_write

False

In [8]:
from data.data_utils import get_datasets

train_dataset, val_dataset = get_datasets(
        cfg.data, split=cfg.split, preprocess=True
    )

Discarded 7 ids


preprocessing temporal data into images: 100%|██████████| 9593/9593 [00:25<00:00, 375.44it/s]
/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/data/data_utils.py:70: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  torch.tensor([x[0] for x in temp]).unsqueeze(1).float()


Discarded 6 ids


preprocessing temporal data into images: 100%|██████████| 2394/2394 [00:06<00:00, 360.51it/s]


# Train/Load Model

In [9]:
from Utils.models import TimEHR

model = TimEHR(cfg)

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[KeOps] Compiling cuda jit compiler engine ... 
[KeOps] Warning : There were warnings or errors :
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore: No such file or directory
g++: error: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: No such file or directory
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore/binders/nvrtc/nvrtc_jit.cpp: No such file or directory

OK
[pyKeOps] Compiling nvrtc binder for python ... 
[KeOps] Warning : There were warnings or errors :
/bin/sh: 1: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: not found
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/keopscore: No such file or directory
g++: error: /mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4.: No such file or directory
g++: error: WGAN-GP/test2/lib/python3.8/site-packages/pykeops/common/keops_io/pykeops_nvrtc.cpp: No such file or directory

OK
Generator has 1,304,964 trainable parameters
Critic has 212,177 trainabl

In [22]:
# model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/lorajx1i')
model.from_pretrained(path_cwgan='hokarami/CWGAN/234t1alr',path_pix2pix='hokarami/PIXGAN/9a0u31rl')


INFO:root:Loading CWGAN from run: hokarami/CWGAN/234t1alr


CWGAN - run_path:  hokarami/CWGAN/234t1alr


INFO:root:Loading PIXGAN from run: hokarami/PIXGAN/9a0u31rl


last_epoch:  150
PIXGAN - run_path:  hokarami/PIXGAN/9a0u31rl
last_epoch:  150


In [ ]:
model.train(train_dataset, val_dataset)

In [23]:
model.cwgan.run_path
model.cwgan.epoch_no

model.pix2pix.run_path
model.pix2pix.epoch_no

model.progress

'hokarami/CWGAN/uj5gf643'

400

'hokarami/PIXGAN/9hec4rye'

150

{}

# Generation

In [23]:
from Utils.utils import mat2df
# get synthetic data
fake_static, fake_data = model.generate(train_dataset, count=len(train_dataset), method='real')
df_ts_fake, df_static_fake = mat2df(fake_data,fake_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get train data
train_static, train_data = model._get_data(train_dataset)
df_ts_train, df_static_train = mat2df(train_data,train_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get test data
val_static, val_data = model._get_data(val_dataset)
df_ts_test, df_static_test = mat2df(val_data,val_static,val_dataset.dynamic_processor,val_dataset.static_processor)

INFO:root:Using static data from the training set (9593 samples)
100%|██████████| 38/38 [00:00<00:00, 355.72it/s]


In [25]:
fake_data.shape

fake_data[0,1,:,:].sum()

(9593, 2, 64, 64)

-3362.0

In [26]:
df_ts_fake

df_ts_train

,ALP,HCO3,SaO2,FiO2,K,Albumin,Urine,Glucose,SysABP,Mg,...,pH,DiasABP,WBC,TroponinT,MAP,HCT,AST,timepoint,id,outcome
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.259169,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,81.508691,NaN,...,NaN,56.546106,NaN,NaN,82.715740,NaN,NaN,1,0,0
2,NaN,17.843637,NaN,0.853452,4.173698,NaN,327.552482,106.033216,115.089550,2.767824,...,7.121957,54.238979,9.822447,NaN,70.102641,34.970562,23.551937,2,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,179.980154,NaN,99.554548,NaN,...,NaN,52.214107,NaN,NaN,70.909316,NaN,NaN,3,0,0
4,NaN,NaN,NaN,0.514076,NaN,NaN,320.432746,NaN,140.899188,NaN,...,NaN,47.589784,NaN,NaN,61.415696,NaN,NaN,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448188,NaN,NaN,NaN,NaN,NaN,NaN,102.418993,NaN,NaN,NaN,...,7.223417,48.959154,NaN,NaN,85.523485,NaN,NaN,41,9592,0
448189,NaN,NaN,NaN,NaN,NaN,NaN,157.941182,NaN,134.170314,NaN,...,NaN,51.360481,NaN,NaN,95.399517,NaN,NaN,43,9592,0
448190,NaN,NaN,NaN,NaN,NaN,NaN,146.094165,NaN,115.050551,NaN,...,NaN,55.061882,NaN,NaN,92.671739,NaN,NaN,45,9592,0
448191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,55.100964,NaN,NaN,NaN,NaN,NaN,46,9592,0


,ALP,HCO3,SaO2,FiO2,K,Albumin,Urine,Glucose,SysABP,Mg,...,pH,DiasABP,WBC,TroponinT,MAP,HCT,AST,timepoint,id,outcome
0,NaN,NaN,NaN,1.0,NaN,NaN,0.000000,NaN,106.000000,NaN,...,7.45,68.999999,7.399999,NaN,81.000000,24.7,NaN,1,0,0
1,NaN,21.0,NaN,0.5,NaN,NaN,45.000000,NaN,105.000002,3.1,...,7.44,62.000001,NaN,NaN,75.999999,28.8,NaN,2,0,0
2,NaN,NaN,99.0,0.5,NaN,NaN,269.999997,NaN,99.000003,NaN,...,7.40,55.999999,NaN,NaN,70.999998,NaN,NaN,3,0,0
3,NaN,NaN,NaN,0.4,NaN,NaN,220.000009,NaN,99.000003,NaN,...,NaN,56.999999,NaN,NaN,69.999999,NaN,NaN,4,0,0
4,NaN,NaN,98.0,NaN,NaN,NaN,99.999993,NaN,80.000000,NaN,...,7.34,55.000001,NaN,NaN,72.000001,28.2,NaN,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.000000,NaN,...,7.36,42.999999,NaN,NaN,57.999999,NaN,NaN,42,9592,0
431757,NaN,17.0,NaN,NaN,4.1,NaN,59.999999,112.000006,122.000000,2.0,...,NaN,50.999999,14.500001,NaN,72.000001,27.5,NaN,43,9592,0
431758,NaN,NaN,NaN,0.5,NaN,NaN,50.000005,NaN,147.000003,NaN,...,NaN,55.999999,NaN,NaN,84.999998,NaN,NaN,45,9592,0
431759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.000001,NaN,...,NaN,56.999999,NaN,NaN,72.000001,NaN,NaN,46,9592,0


# Evaluation

In [27]:
from Utils.evaluation import evaluate

inputs = {

    # normalized data
    'fake_static': fake_static,
    'fake_data': fake_data,
    'train_static': train_static,
    'train_data': train_data,
    'test_static': val_static,
    'test_data': val_data,

    # dataframes from normalized data
    'df_ts_fake': df_ts_fake,
    'df_static_fake': df_static_fake,
    'df_ts_train': df_ts_train,
    'df_static_train': df_static_train,
    'df_ts_test': df_ts_test,
    'df_static_test': df_static_test,

    'state_vars': train_dataset.temporal_features,
}

In [28]:
fake_data.shape, train_data.shape, val_data.shape

((9593, 2, 64, 64), (9593, 2, 64, 64), (2394, 2, 64, 64))

In [29]:
wandb_task_name = f"NOTEBOOK-{cfg.data.name}-s{cfg.split}"

evaluate(inputs, wandb_task_name)

# You can now view the results in the wandb dashboard under USERNAME/TimEHR_Results
# The results are also available locally in Results/{wandb_task_name}/wandb/{run_name}/

wandb: WARNING Path ./Results/NOTEBOOK-p12-s0/TimEHR-Eval/wandb/ wasn't writable, using system temp directory.


wandb: WARNING Path ./Results/NOTEBOOK-p12-s0/TimEHR-Eval/wandb/ wasn't writable, using system temp directory


[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 
[info] Preprocessing 

[info] TRTR
[LightGBM] [Info] Number of positive: 1376, number of negative: 8217
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020874 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28291
[LightGBM] [Info] Number of data points in the train set: 9593, number of used features: 175
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.143438 -> initscore=-1.787024
[LightGBM] [Info] Start training from score -1.787024
AUROC: 0.8316548363996352
AUPRC: 0.4755285238145955
F1: 0.41573033707865165

[info] TSTR
[LightGBM] [Info] Number of positive: 1376, number of negative: 8217
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41877
[LightGBM] [Info] Number of data points in the trai

Adv ACC/AA_test_syn,▁
Adv ACC/AA_train_syn,▁
Adv ACC/AA_train_test,▁
Adv ACC/NNAA,▁
MIA/JSD,▁
MIA/WD,▁
MIA/knn-auroc,▁
Synthcity[Train-Synthetic]/privacy.identifiability_score.score,▁
Synthcity[Train-Synthetic]/privacy.identifiability_score.score_OC,▁
Synthcity[Train-Synthetic]/stats.prdc.coverage,▁
Synthcity[Train-Synthetic]/stats.prdc.density,▁
